In [6]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_boston,load_diabetes,load_iris,load_wine
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ridge_regression,lasso_path

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

import seaborn as sns
import pickle
import json

In [7]:
df = pd.read_csv('medical_insurance.csv')
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [9]:
df['sex'].value_counts()

male      676
female    662
Name: sex, dtype: int64

In [10]:
df['sex'].replace({'male' : 0, 'female' : 1},inplace = True)

In [11]:
df['smoker'].value_counts()

no     1064
yes     274
Name: smoker, dtype: int64

In [12]:
df['smoker'].replace({'yes' : 0, 'no' : 1},inplace = True)

In [13]:
df['region'].value_counts()

southeast    364
southwest    325
northwest    325
northeast    324
Name: region, dtype: int64

In [14]:
df = pd.get_dummies(df,columns =['region'],prefix = 'reg')      # reg_southeast

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1338 non-null   int64  
 1   sex            1338 non-null   int64  
 2   bmi            1338 non-null   float64
 3   children       1338 non-null   int64  
 4   smoker         1338 non-null   int64  
 5   charges        1338 non-null   float64
 6   reg_northeast  1338 non-null   uint8  
 7   reg_northwest  1338 non-null   uint8  
 8   reg_southeast  1338 non-null   uint8  
 9   reg_southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


In [16]:
df.isna().sum()

age              0
sex              0
bmi              0
children         0
smoker           0
charges          0
reg_northeast    0
reg_northwest    0
reg_southeast    0
reg_southwest    0
dtype: int64

In [17]:
x = df.drop('charges',axis = 1)
y = df['charges']

In [18]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state= 50)

In [19]:
x_train

,age,sex,bmi,children,smoker,reg_northeast,reg_northwest,reg_southeast,reg_southwest
1048,25,1,22.515,1,1,0,1,0,0
45,55,0,37.300,0,1,0,0,0,1
201,48,1,32.230,1,1,0,0,1,0
153,42,1,23.370,0,0,1,0,0,0
1251,19,0,19.800,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...
229,47,0,25.460,2,1,1,0,0,0
70,27,1,24.750,0,0,0,0,1,0
132,53,1,35.900,2,1,0,0,0,1
1313,19,1,34.700,2,0,0,0,0,1


In [20]:
x_test

,age,sex,bmi,children,smoker,reg_northeast,reg_northwest,reg_southeast,reg_southwest
589,38,1,30.69,1,1,0,0,1,0
383,35,1,43.34,2,1,0,0,1,0
534,64,0,40.48,0,1,0,0,1,0
284,52,1,31.20,0,1,0,0,0,1
821,26,0,17.67,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...
871,26,1,29.48,1,1,0,0,1,0
496,31,1,23.60,2,1,0,0,0,1
578,52,0,30.20,1,1,0,0,0,1
1012,61,1,33.33,4,1,0,0,1,0


In [21]:
y_train

1048     3594.17085
45      20630.28351
201      8871.15170
153     19964.74630
1251     1241.56500
           ...     
229      9225.25640
70      16577.77950
132     11163.56800
1313    36397.57600
109     47055.53210
Name: charges, Length: 1070, dtype: float64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1338 non-null   int64  
 1   sex            1338 non-null   int64  
 2   bmi            1338 non-null   float64
 3   children       1338 non-null   int64  
 4   smoker         1338 non-null   int64  
 5   charges        1338 non-null   float64
 6   reg_northeast  1338 non-null   uint8  
 7   reg_northwest  1338 non-null   uint8  
 8   reg_southeast  1338 non-null   uint8  
 9   reg_southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


In [23]:
model = LinearRegression()
model.fit(x_train,y_train)

LinearRegression()

In [24]:
y_pred_test = model.predict(x_test)

mse = mean_squared_error(y_test,y_pred_test)
print("MSE :",mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

r2 = r2_score(y_test,y_pred_test)
print("R-square :",r2)


MSE : 34742386.89805868
RMSE : 5894.267291025974
R-square : 0.7832551496410705


In [25]:
y_pred_train = model.predict(x_train)

mse = mean_squared_error(y_train,y_pred_train)
print("MSE :",mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

r2 = r2_score(y_train,y_pred_train)
print("R-square :",r2)

MSE : 37060086.19885744
RMSE : 6087.699581850064
R-square : 0.7407239225747873


In [26]:
x.head(1).T

,0
age,19.0
sex,1.0
bmi,27.9
children,0.0
smoker,0.0
reg_northeast,0.0
reg_northwest,0.0
reg_southeast,0.0
reg_southwest,1.0


In [27]:
age= 19.0
sex= 'female'
bmi= 77.9
children= 0.0
smoker= 'no'
region = 'northeast'

In [28]:
project_data = {'sex' : {'male' : 0 , 'female' : 1},
                     'smoker' : {'yes' : 0 , 'no' : 1},
               'columns' }
project_data

{'sex': {'male': 0, 'female': 1}, 'smoker': {'yes': 0, 'no': 1}}

In [29]:
column_names = x.columns
column_names

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'reg_northeast',
       'reg_northwest', 'reg_southeast', 'reg_southwest'],
      dtype='object')

In [30]:
region =  'reg_' + region                        
region_index = np.where(column_names == region)[0][0]

In [31]:
region_index

5

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   age            1338 non-null   int64  
 1   sex            1338 non-null   int64  
 2   bmi            1338 non-null   float64
 3   children       1338 non-null   int64  
 4   smoker         1338 non-null   int64  
 5   charges        1338 non-null   float64
 6   reg_northeast  1338 non-null   uint8  
 7   reg_northwest  1338 non-null   uint8  
 8   reg_southeast  1338 non-null   uint8  
 9   reg_southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


In [33]:
array1 = np.zeros(len(x.columns),dtype = int)
array1

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [34]:
array1[0] = age
array1[1] = label_encoded_data['sex'][sex]
array1[2] = bmi
array1[3] = children 
array1[4] = label_encoded_data['smoker'][smoker]
array1[5] = region_index = 1
array1

array([19,  1, 77,  0,  1,  1,  0,  0,  0])

In [35]:
model.predict([array1])[0]

C:\Users\surabh\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


18783.541855431824

In [37]:
with open('Model_Evaluation.pkl','wb') as f:
    pickle.dump(model,f)

In [ ]:
with open('project_data.json','w') as f:
    json.dump(project_data,f)